## Solving a standard form Linear Programming  problem
Let us try to write the JuMP code for the following standard form optimization problem:

$$
\begin{align}
& \text{minimize} && c^T x \\
& \text{subject to} && A x = b \\
&                   && x \succeq 0 \\
&                   && x \in \mathbb{R}^n
\end{align}
$$


where, $n = 4$, $c=(1, 3, 5, 2)$, $A = \begin{pmatrix}
  1 & 1 & 9 & 5 \\
  3 & 5 & 0 & 8 \\
  2 & 0 & 6 & 13
 \end{pmatrix}$ and $b=(7, 3, 5)$. The symbol $\succeq$ ($\preceq$) stands for element-wise greater (less) than or equal to.
 
Notice that here, defining variables one by one would be extremely tedious.  Thankfully we can make use of Julia and JuMP's array capabilities to create sets of variables all at once.  


# Representing vectors in Julia
 
 * A column vector, $y=(y_1, y_2, \ldots, y_n)= \begin{pmatrix}
  y_1 \\
   y_2 \\
   . \\
   . \\
   y_n
 \end{pmatrix} \in \mathbb{R}^n$ will be written in Julia as `[y[1];y[2];...;y[n]]`. 
 
 For example to create column vector $\begin{pmatrix}
  3 \\
   2.4 \\
   9.1 \\
 \end{pmatrix}$  use: `[3; 2.4; 9.1]`.

In [2]:
[3; 2.4; 9.1] # Column vector

3-element Array{Float64,1}:
 3.0
 2.4
 9.1

* A row vector, $z=(z_1 \; z_2 \; \ldots \; z_n) \in \mathbb{R}^{1 \times n}$ will be written in Julia as `[z[1] y[2]...z[n]]`. 
 
 For example to create row vector $(1.2 \; 3.5 \; 8.21)$  use: `[1.2 3.5 8.21]`.

In [3]:
[1.2 3.5 8.21] # Row vector

1×3 Array{Float64,2}:
 1.2  3.5  8.21

* To create a $m \times n$ matrix 

$$
A = \begin{pmatrix}
  A_{11} & A_{12} & A_{13} & \ldots &A_{1n} \\
  \ldots & \ldots & \ldots & \ldots & \ldots  \\
  A_{m1} & A_{m2} & A_{m3} & \ldots & A_{mn}
 \end{pmatrix}
$$

write:
 
 `[A[1,1] A[1,2] A[1,3]... A[1,n];`
 ` ... ; `
  `A[m,1] A[m,2] ... A[m,n]]`.
  
  So the matrix 
  
  $$
  A = \begin{pmatrix}
  1 & 1 & 9 & 5 \\
  3 & 5 & 0 & 8 \\
  2 & 0 & 6 & 13
 \end{pmatrix}
 $$ 
 
 is represented in Julia as: 
 
  `A= [
     1 1 9 5;
     3 5 0 8;
     2 0 6 13
    ]`

In [4]:
# Generating a matrix
A= [
     1 1 9 5;
     3 5 0 8;
     2 0 6 13
    ]

3×4 Array{Int64,2}:
 1  1  9   5
 3  5  0   8
 2  0  6  13

  $A_{ij}$ can be accessed by `A[i,j]` ,the $i$th row of the matrix A is represented by `A[i,:]`, the $j$th column of the matrix A is represented by `A[:,j]`. 
  
  The size of a matrix $A$ can be determined by running the command `size(A)`. If we write `numRows, numCols = size(A)`, then `numRows` and `numCols` will contain the total number of rows and columns of A respectively.

In [5]:
numRows, numCols = size(A)
println(
"A has ", numRows, " rows and ", numCols, " columns \n",
"A[3,3] is ", A[3,3], "\n",
"The 3rd column of A is ", A[:,3], "\n",
"The 2nd row of A is ", A[2,:]
)


A has 3 rows and 4 columns 
A[3,3] is 6
The 3rd column of A is [9, 0, 6]
The 2nd row of A is [3, 5, 0, 8]


Suppose $x,y \in \mathbb{R}^n$. Then $x^T y =\sum_{i=1}^{n} {x_i y_i}$ is written as `dot(x,y)`. 

In [11]:
y=[1; 2; 3; 4]
x=[5; 6; 7; 8]
xTy=dot(x,y)
println("dot(x,y) = ",xTy)
#Or equivalently...
xTy = x'y
println("x'y = ",xTy)

dot(x,y) = 70
x'y = 70


# Structure of a JuMP model

Any JuMP model that describes an optimization problem must have four parts: 

- **Model Object**, 
- **Variables**, 
- **Objective**, 
- **Constraints**.

## Model

Any instance of an optimization problem corresponds to a model object. This model object is associated with all the variables, constraints and objective of the instance. It is constructed using `modelName = Model(solver=`*solver of our preference*`)`. If no solver is specified, then `ClpSolver()` and/or `CbcSolver()` will be used by default. Here `modelName` is any valid name. We will limit ourselves in the open source solvers such as:

* Open Source Linear Programming Solver: `ClpSolver(), GLPKSolverLP()`
* Open Source Mixed Integer Programming Solver: `GLPKSolverMIP() CbcSolver() GurobiSolver() CPLEXSolver()`
* Commercial Mixed Integer Programming Solver: `GurobiSolver() CPLEXSolver()`
* Open Source Nonlinear Programming Solver: `IpoptSolver()`

In [13]:
using JuMP
myModel = Model() # ClpSolver() and/or CbcSolver() will be used based on the problem

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is default solver

## Variables
Variables are defined using `@variable` macro, which takes up to three input arguments. The *first* argument is the name of the model. Then the *second* argument contains the name of the variable, and a bound on the variable if it exists. The *third* argument is not needed if the variable is real. When the variable is binary or integer, then `Bin` or `Int`, respectively, is used in place of the third argument.


### Examples of Variables
Suppose the model object is `myModel`. 

- To describe a variable $z \in \mathbb{R}$ such that $0 \leq z \leq 10$
write

In [14]:
@variable(myModel, 0 <= z <= 10)

z

- Now consider a decision variable $x \in \mathbb{R}^n$, and it has a bound $l \preceq x \preceq u$, where naturally $l, u \in \mathbb{R}^n$.  For that we write <br>

In [15]:
# INPUT DATA, CHANGE THEM TO YOUR REQUIREMENT
#-------------------------------------------
n = 10
l = [1; 2; 3; 4; 5; 6; 7; 8; 9; 10]
u = [10; 11; 12; 13; 14; 15; 16; 17; 18; 19]

10-element Array{Int64,1}:
 10
 11
 12
 13
 14
 15
 16
 17
 18
 19

In [16]:
@variable(myModel, l[i] <= x[i=1:n] <= u[i])

10-element Array{JuMP.Variable,1}:
 x[1] 
 x[2] 
 x[3] 
 x[4] 
 x[5] 
 x[6] 
 x[7] 
 x[8] 
 x[9] 
 x[10]

- Suppose we have decision variables $x \in \mathbb{R}^n$, $y \in \mathbb{Z}^m$ and $z \in \mathbb \{0,1\}^p$ such that $x \succeq 0$, $a \preceq y \preceq b$. Here $a, b \in \mathbb{Z}^m$. To express this in JuMP we write

In [18]:
# INPUT DATA, CHANGE THEM TO YOUR REQUIREMENT
#-------------------------------------------
n = 4 # dimension of x
m = 3 # dimension of y
p = 2 # dimensin of z
a = [0; 1; 2]
b = [3; 4; 7]

3-element Array{Int64,1}:
 3
 4
 7

In [19]:
# VARIABLE DEFINITION
# -------------------
@variable(myModel, x2[i=1:n] >= 0)

4-element Array{JuMP.Variable,1}:
 x[1]
 x[2]
 x[3]
 x[4]

In [20]:
@variable(myModel, a[i] <= y[i=1:m] <= b[i], Int)

3-element Array{JuMP.Variable,1}:
 y[1]
 y[2]
 y[3]

In [21]:
@variable(myModel, z2[i=1:p], Bin)

2-element Array{JuMP.Variable,1}:
 z[1]
 z[2]

## Constraints
Constraints are added by using `@constraint` macro. The first argument is the model object the constraint is associated with, the second argument is the reference to that constraint and the third argument is the constraint description. The constraint reference comes handy when we want to manipulate the constraint later or access the dual variables associated with it. If no constraint reference is needed, then the second argument is the constraint description. <br>

### Examples of Constraints
Let's give some examples on writing constraints in JuMP. Suppose the model name is `yourModel`.

In [22]:
yourModel = Model()

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is default solver

1. Consider variables $x, y \in \mathbb{R}$ which are coupled by the constraints $5 x +3 y \leq 5$. We write this as <br>
`@constraint(yourModel, 5*x + 3*y <= 5)` <br>
Naturally, `x` and `y` have to be defined first using `@variable` macro.

In [23]:
@variable(yourModel, x)
@variable(yourModel, y)
@constraint(yourModel, 5*x + 3*y <= 5)

5 x + 3 y ≤ 5

 Here no constraint reference is given. Now suppose we want to get the dual value of some constraint after solving the problem, then we would need a constraint reference to assign to the constraint first. Let's call the constraint reference as `conRef1` (it could be any valid name). Then the same constraint have to be written as: <br>
`@constraint(yourModel, conRef1, 6*x + 4*y >= 5)` <br>
When we would need the dual value after solving the problem we just write `println(getDual(conRef1))`. <br>

In [24]:
@constraint(yourModel, conRef1, 6*x + 4*y >= 5) 

6 x + 4 y ≥ 5

2. Consider a variable $x \in \mathbb{R}^4$, a coefficient vector $a=(1, -3, 5, -7)$ We want to write a constraint of the form $\sum_{i=1}^4{a_i x_i} \leq 3$. In JuMP we write:<br>

In [25]:
a = [1; -3; 5; 7] 
@variable(yourModel, w[1:4])
@constraint(yourModel, sum(a[i]*w[i] for i in 1:4) <= 3)

w[1] - 3 w[2] + 5 w[3] + 7 w[4] ≤ 3

## Objective
Objective is set using `@objective` macro. It has three arguments. The first argument is as usual the model object. The second one is either `Max` if we want to maximize the objective function, or `Min` when we want to minimize. The last argument is the description of the objective which has similar syntax to that of constraint definition.

### Example of objective
For the previous model, consider the decision variable $w \in \mathbb{R}^4$ and cost vector $c = (2, 3 , 4, 5)$. We want to minimize $c^T w$. In JuMP we would write:

In [27]:
c = [2; 3; 4; 5] 
@objective(yourModel, Min, sum(c[i]*w[i] for i in 1:4))

2 w[1] + 3 w[2] + 4 w[3] + 5 w[4]

# Back to our Linear Program
$$
\begin{align}
& \text{minimize} && c^T x \\
& \text{subject to} && A x = b \\
&                   && x \succeq 0 \\
&                   && x \in \mathbb{R}^n
\end{align}
$$


where, $n = 4$, $c=(1, 3, 5, 2)$, $A = \begin{pmatrix}
  1 & 1 & 9 & 5 \\
  3 & 5 & 0 & 8 \\
  2 & 0 & 6 & 13
 \end{pmatrix}$ and $b=(7, 3, 5)$. The symbol $\succeq$ ($\preceq$) stands for element-wise greater (less) than or equal to.

In [28]:
using JuMP  # Need to say it whenever we use JuMP

using GLPKMathProgInterface # Loading the package for using the GLPK solver


INFO: Precompiling module GLPK.
ERROR: LoadError: GLPK not properly installed. Please run Pkg.build("GLPK")
Stacktrace:
 [1] error(::String) at ./error.jl:21
 [2] include_from_node1(::String) at ./loading.jl:569
 [3] include(::String) at ./sysimg.jl:14
 [4] anonymous at ./<missing>:2
while loading /home/jordan/.julia/v0.6/GLPK/src/GLPK.jl, in expression starting on line 198


LoadError: LoadError: LoadError: [91mFailed to precompile GLPK to /home/jordan/.julia/lib/v0.6/GLPK.ji.[39m
while loading /home/jordan/.julia/v0.6/GLPKMathProgInterface/src/GLPKInterfaceBase.jl, in expression starting on line 4
while loading /home/jordan/.julia/v0.6/GLPKMathProgInterface/src/GLPKMathProgInterface.jl, in expression starting on line 9

In [29]:
#MODEL CONSTRUCTION
#------------------

sfLpModel = Model(solver=GLPKSolverLP()) # Name of the model object

LoadError: [91mUndefVarError: GLPKSolverLP not defined[39m

In [30]:
#INPUT DATA
#----------

c = [1; 3; 5; 2] 

A= [
     1 1 9 5;
     3 5 0 8;
     2 0 6 13
    ]

b = [7; 3; 5] 

m, n = size(A) # m = number of rows of A, n = number of columns of A

(3, 4)

In [31]:
#VARIABLES
#---------

@variable(sfLpModel, x[1:n] >= 0) # Models x >=0


LoadError: [91mUndefVarError: sfLpModel not defined[39m

In [32]:
#CONSTRAINTS
#-----------

for i in 1:m # for all rows do the following
    @constraint(sfLpModel, sum(A[i,j]*x[j] for j in 1:n) == b[i]) # the ith row 
    # of A*x is equal to the ith component of b
end # end of the for loop

LoadError: [91mUndefVarError: sfLpModel not defined[39m

In [33]:
#OBJECTIVE
#---------

@objective(sfLpModel, Min, sum(c[j]*x[j] for j in 1:n)) # minimize c'x 

LoadError: [91mUndefVarError: sfLpModel not defined[39m

In [34]:
#THE MODEL IN A HUMAN-READABLE FORMAT
#------------------------------------

println("The optimization problem to be solved is:")
print(sfLpModel) # Shows the model constructed in a human-readable form

The optimization problem to be solved is:


LoadError: [91mUndefVarError: sfLpModel not defined[39m

In [35]:
status = solve(sfLpModel) # solves the model  

LoadError: [91mUndefVarError: sfLpModel not defined[39m

In [ ]:
#SOLVE IT AND DISPLAY THE RESULTS
#--------------------------------

println("Objective value: ", getObjectiveValue(sfLpModel)) # getObjectiveValue(model_name) gives the optimum objective value

println("Optimal solution is x = \n", getValue(x)) # getValue(decision_variable) will give the optimum value 
                                                   # of the associated decision variable
